In [1]:
import os
cuda_path = 'C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v10.0\\bin'
cudnn_path = 'D:\\Work\\software\\cuda\\bin'
os.environ["PATH"] += os.pathsep + cuda_path
os.environ["PATH"] += os.pathsep + cudnn_path
import keras
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
sess = tf.Session(config=config)
set_session(sess)

Using TensorFlow backend.
C:\Users\Richard\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Richard\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Richard\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Richard\Anaconda3\lib\site-packages\tensorflow\python\framework\d

In [2]:
#import analysis
import sys
sys.path.insert(0,'../scripts/')

import test_features
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots,scatter
import seaborn as sns
from sklearn.externals import joblib
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.model_selection import cross_val_score
import os
#import analysis
def read_data(file_dir,DB_file_dir):
    
    data = []
    label = []
    
    print('loading..{}'.format(file_dir))
    df=pd.read_pickle(file_dir)
    psm_ID=list(pd.read_csv(DB_file_dir)['Scan'])
    y=np.zeros(df.shape[0])

    for i in range(0,df.shape[0]):
        if df.index[i] in psm_ID:
            y[i]=1

    X = df.drop('RT',axis=1).fillna(0).values
    y =y.astype(int)[X.sum(axis=1)!=0]
    X = X[X.sum(axis=1)!=0]
    X = X/X.sum(axis=1).reshape(X.shape[0],1) #normalization
    data.append(X)
    label.append(y)
    del X

    data = np.concatenate((data))
    label = np.concatenate((label))
    return data,label

from numpy.random import randint,seed,choice
def under_sampler(data,label,n_sample):
    
    index_maj = np.where(label==0)[0]
    index_min = np.where(label==1)[0]
    
    seed(19)
    sample_index = choice(len(index_maj),size=n_sample,replace=False)
    index_maj_sampled = index_maj[sample_index]
    data_sampled = data[np.concatenate((index_maj_sampled,index_min))]
    label_sampled = label[np.concatenate((index_maj_sampled,index_min))]
    
    shuffle_index = choice(len(data_sampled),size=len(data_sampled),replace=False)
    data_sampled = data_sampled[shuffle_index]
    label_sampled = label_sampled[shuffle_index]
    
    return data_sampled,label_sampled

def load_sample_data(file_dir,DB_file_dir,n_sample):
    
    X_train,X_test,Y_train,Y_test = [],[],[],[]
    for i in range(len(file_dir)):
        data,label = read_data(file_dir[i],DB_file_dir[i])
        x_train, x_test, y_train, y_test = train_test_split(data, label, test_size=0.3, random_state=1)
        del data
        
        x_train_sampled,y_train_sampled = under_sampler(x_train,y_train,len(y_train[y_train==1])*n_sample)
        del x_train,y_train
        X_train.append(x_train_sampled)
        X_test.append(x_test)
        Y_train.append(y_train_sampled)
        Y_test.append(y_test)
        del x_test,y_test,x_train_sampled,y_train_sampled
        
    X_train = np.concatenate(X_train)
    X_test = np.concatenate(X_test)
    Y_train = np.concatenate(Y_train)
    Y_test = np.concatenate(Y_test)

    return X_train,X_test,Y_train,Y_test

C:\Users\Richard\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
# file_dir = ['../Orbitrap_aplysia/abd.pkl','../Orbitrap_aplysia/buc.pkl','../Orbitrap_aplysia/cer.pkl']
# DB_file_dir = ['../Aplysia_ganglia/Abdominal/DB search psm.csv','../Aplysia_ganglia/Buccal/DB search psm.csv',
#               '../Aplysia_ganglia/Cerebral/DB search psm.csv']

# X_train,X_test,Y_train,Y_test = load_sample_data(file_dir,DB_file_dir,2)

loading..../Orbitrap_aplysia/abd.pkl
loading..../Orbitrap_aplysia/buc.pkl
loading..../Orbitrap_aplysia/cer.pkl


In [3]:
X_train = np.load('../Orbitrap_aplysia/train_data_combined_sampled_x2.npy')
Y_train = np.load('../Orbitrap_aplysia/train_label_combined_sampled_x2.npy')
X_test = np.load('../Orbitrap_aplysia/cer_test_data.npy')
Y_test = np.load('../Orbitrap_aplysia/cer_test_label.npy')

In [4]:
from numpy.random import randint,seed,choice
seed(19)
shuffle_index = choice(len(X_train),size=len(X_train),replace=False)
X_train = X_train[shuffle_index]
Y_train =Y_train[shuffle_index]

In [11]:
X_train=X_train.reshape((X_train.shape[0],X_train.shape[1]))
X_test=X_test.reshape((X_test.shape[0],X_test.shape[1]))

In [6]:
from __future__ import print_function
from keras.layers import Dense, Conv1D, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling1D, AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os

# from keras.backend.tensorflow_backend import set_session
# import tensorflow as tf
# K.clear_session()
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
# config.log_device_placement = True  # to log device placement (on which device the operation ran)
# sess = tf.Session(config=config)
# set_session(sess)  # set this TensorFlow session as the default session for Keras


# Training parameters
batch_size = 32  # orig paper trained all networks with batch_size=128
epochs = 100
data_augmentation = False
num_classes = 2
input_shape = X_train.shape[1:]
scale_weight = Y_train[Y_train==0].shape[0]/Y_train[Y_train==1].shape[0]

n = 1
version = 2
# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version)

# Input image dimensions.
# Convert class vectors to binary class matrices.
#y_train_aug = keras.utils.to_categorical(y_train_aug, num_classes)
Y_train_ = keras.utils.to_categorical(Y_train, num_classes)
Y_test_ = keras.utils.to_categorical(Y_test, num_classes)

In [7]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-2
    if epoch > 100:
        lr *= 0.5e-6
    elif epoch > 80:
        lr *= 1e-6
    elif epoch > 60:
        lr *= 1e-5
    elif epoch > 40:
        lr *= 1e-4
    print('Learning rate: ', lr)
    return lr


def resnet_layer(inputs,
                 num_filters=8,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv1D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

def resnet_v2(input_shape, depth, num_classes=2):
    """ResNet Version 2 Model builder [b]

    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.


    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv1D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling1D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [8]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

INTERESTING_CLASS_ID =  1 # Choose the class of interest

def single_class_accuracy(y_true, y_pred):
    class_id_true = K.argmax(y_true, axis=-1)
    class_id_preds = K.argmax(y_pred, axis=-1)
    accuracy_mask = K.cast(K.equal(class_id_preds, INTERESTING_CLASS_ID), 'int32')
    #recall_mask = K.cast(K.equal(class_id_true, INTERESTING_CLASS_ID), 'int32')

    class_acc_tensor = K.cast(K.equal(class_id_true, class_id_preds), 'int32') * accuracy_mask
    #class_recall_tensor = K.cast(K.equal(class_id_true, class_id_true), 'int32') * recall_mask
    class_acc = K.sum(class_acc_tensor) / K.maximum(K.sum(accuracy_mask), 1)
    #class_recall = K.sum(class_recall_tensor) / K.maximum(K.sum(recall_mask), 1)
    return class_acc

def single_class_recall(y_true, y_pred):
    class_id_true = K.argmax(y_true, axis=-1)
    class_id_preds = K.argmax(y_pred, axis=-1)
    #accuracy_mask = K.cast(K.equal(class_id_preds, INTERESTING_CLASS_ID), 'int32')
    recall_mask = K.cast(K.equal(class_id_true, INTERESTING_CLASS_ID), 'int32')

    #class_acc_tensor = K.cast(K.equal(class_id_true, class_id_preds), 'int32') * accuracy_mask
    class_recall_tensor = K.cast(K.equal(class_id_true, class_id_preds), 'int32') * recall_mask
    #class_acc = K.sum(class_acc_tensor) / K.maximum(K.sum(accuracy_mask), 1)
    class_recall = K.sum(class_recall_tensor) / K.maximum(K.sum(recall_mask), 1)
    return class_recall

if version == 2:
    model = resnet_v2(input_shape=input_shape, depth=depth)
else:
    model = resnet_v1(input_shape=input_shape, depth=depth)

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adadelta(lr=1e-3),
                  metrics=[recall_m,precision_m,f1_m,single_class_accuracy,single_class_recall])
model.summary()
print(model_type)

W0117 00:07:06.220641  9492 deprecation_wrapper.py:119] From C:\Users\Richard\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0117 00:07:06.221638  9492 deprecation_wrapper.py:119] From C:\Users\Richard\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0117 00:07:06.257993  9492 deprecation_wrapper.py:119] From C:\Users\Richard\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0117 00:07:06.367401  9492 deprecation_wrapper.py:119] From C:\Users\Richard\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0117 00:07:06.968178  9492

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 12001, 1)     0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 12001, 16)    64          input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 12001, 16)    64          conv1d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 12001, 16)    0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv1d_2 (

In [9]:
save_dir = os.path.join(os.getcwd(), 'models')
model_name = 'Combined_Sampled_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_single_class_accuracy',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer =  ReduceLROnPlateau(min_lr=1e-6, monitor='val_single_class_accuracy', verbose=1,factor=0.8,patience=20)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

# Run training, with or without data augmentation.
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(X_train, Y_train_,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, Y_test_),
              shuffle=True,
              callbacks=callbacks,class_weight={0:1,1:scale_weight})

Not using data augmentation.


W0117 00:07:08.433354  9492 deprecation.py:323] From C:\Users\Richard\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 28548 samples, validate on 13627 samples
Epoch 1/100
Learning rate:  0.01
28548/28548 [==============================] - 152s 5ms/step - loss: 0.9131 - recall_m: 0.7608 - precision_m: 0.7608 - f1_m: 0.7608 - single_class_accuracy: 0.6190 - single_class_recall: 0.7954 - val_loss: 0.7701 - val_recall_m: 0.7982 - val_precision_m: 0.7982 - val_f1_m: 0.7982 - val_single_class_accuracy: 0.2813 - val_single_class_recall: 0.8029

Epoch 00001: val_single_class_accuracy improved from -inf to 0.28133, saving model to D:\Work\projects\DL for MS\Notebooks\models\Combined_Sampled_ResNet11v2_model.001.h5
Epoch 2/100
Learning rate:  0.01
28548/28548 [==============================] - 144s 5ms/step - loss: 0.7824 - recall_m: 0.8305 - precision_m: 0.8305 - f1_m: 0.8305 - single_class_accuracy: 0.7053 - single_class_recall: 0.8516 - val_loss: 0.6591 - val_recall_m: 0.8320 - val_precision_m: 0.8320 - val_f1_m: 0.8320 - val_single_class_accuracy: 0.3161 - val_single_class_recall: 0.7684

Epoch 000

28548/28548 [==============================] - 140s 5ms/step - loss: 0.6159 - recall_m: 0.8943 - precision_m: 0.8943 - f1_m: 0.8943 - single_class_accuracy: 0.8027 - single_class_recall: 0.9085 - val_loss: 0.7976 - val_recall_m: 0.7815 - val_precision_m: 0.7815 - val_f1_m: 0.7815 - val_single_class_accuracy: 0.2779 - val_single_class_recall: 0.8924

Epoch 00018: val_single_class_accuracy did not improve from 0.38746
Epoch 19/100
Learning rate:  0.01
28548/28548 [==============================] - 140s 5ms/step - loss: 0.6101 - recall_m: 0.8979 - precision_m: 0.8979 - f1_m: 0.8979 - single_class_accuracy: 0.8083 - single_class_recall: 0.9119 - val_loss: 2.3561 - val_recall_m: 0.3247 - val_precision_m: 0.3247 - val_f1_m: 0.3247 - val_single_class_accuracy: 0.1117 - val_single_class_recall: 0.9370

Epoch 00019: val_single_class_accuracy did not improve from 0.38746
Epoch 20/100
Learning rate:  0.01
28548/28548 [==============================] - 140s 5ms/step - loss: 0.6046 - recall_m: 0.89

28548/28548 [==============================] - 140s 5ms/step - loss: 0.5476 - recall_m: 0.9143 - precision_m: 0.9143 - f1_m: 0.9143 - single_class_accuracy: 0.8354 - single_class_recall: 0.9286 - val_loss: 0.6262 - val_recall_m: 0.8612 - val_precision_m: 0.8612 - val_f1_m: 0.8612 - val_single_class_accuracy: 0.3684 - val_single_class_recall: 0.8534

Epoch 00035: val_single_class_accuracy did not improve from 0.41000
Epoch 36/100
Learning rate:  0.01
28548/28548 [==============================] - 140s 5ms/step - loss: 0.5419 - recall_m: 0.9165 - precision_m: 0.9165 - f1_m: 0.9165 - single_class_accuracy: 0.8375 - single_class_recall: 0.9301 - val_loss: 0.7150 - val_recall_m: 0.8223 - val_precision_m: 0.8223 - val_f1_m: 0.8223 - val_single_class_accuracy: 0.3187 - val_single_class_recall: 0.8792

Epoch 00036: val_single_class_accuracy did not improve from 0.41000
Epoch 37/100
Learning rate:  0.01
28548/28548 [==============================] - 140s 5ms/step - loss: 0.5389 - recall_m: 0.91


Epoch 00052: val_single_class_accuracy did not improve from 0.41000
Epoch 53/100
Learning rate:  1.0000000000000002e-06
28548/28548 [==============================] - 140s 5ms/step - loss: 0.5208 - recall_m: 0.9193 - precision_m: 0.9193 - f1_m: 0.9193 - single_class_accuracy: 0.8299 - single_class_recall: 0.9546 - val_loss: 0.6221 - val_recall_m: 0.8586 - val_precision_m: 0.8586 - val_f1_m: 0.8586 - val_single_class_accuracy: 0.3681 - val_single_class_recall: 0.8594

Epoch 00053: val_single_class_accuracy did not improve from 0.41000
Epoch 54/100
Learning rate:  1.0000000000000002e-06
28548/28548 [==============================] - 140s 5ms/step - loss: 0.5214 - recall_m: 0.9181 - precision_m: 0.9181 - f1_m: 0.9181 - single_class_accuracy: 0.8268 - single_class_recall: 0.9543 - val_loss: 0.6237 - val_recall_m: 0.8581 - val_precision_m: 0.8581 - val_f1_m: 0.8581 - val_single_class_accuracy: 0.3674 - val_single_class_recall: 0.8602

Epoch 00054: val_single_class_accuracy did not improve 

28548/28548 [==============================] - 140s 5ms/step - loss: 0.5193 - recall_m: 0.9201 - precision_m: 0.9201 - f1_m: 0.9201 - single_class_accuracy: 0.8322 - single_class_recall: 0.9502 - val_loss: 0.6174 - val_recall_m: 0.8598 - val_precision_m: 0.8598 - val_f1_m: 0.8598 - val_single_class_accuracy: 0.3699 - val_single_class_recall: 0.8586

Epoch 00070: val_single_class_accuracy did not improve from 0.41000
Epoch 71/100
Learning rate:  1.0000000000000001e-07
28548/28548 [==============================] - 140s 5ms/step - loss: 0.5190 - recall_m: 0.9201 - precision_m: 0.9201 - f1_m: 0.9201 - single_class_accuracy: 0.8321 - single_class_recall: 0.9522 - val_loss: 0.6193 - val_recall_m: 0.8600 - val_precision_m: 0.8600 - val_f1_m: 0.8600 - val_single_class_accuracy: 0.3701 - val_single_class_recall: 0.8586

Epoch 00071: val_single_class_accuracy did not improve from 0.41000
Epoch 72/100
Learning rate:  1.0000000000000001e-07
28548/28548 [==============================] - 140s 5ms/


Epoch 00087: val_single_class_accuracy did not improve from 0.41000
Epoch 88/100
Learning rate:  1e-08
28548/28548 [==============================] - 140s 5ms/step - loss: 0.5199 - recall_m: 0.9200 - precision_m: 0.9200 - f1_m: 0.9200 - single_class_accuracy: 0.8326 - single_class_recall: 0.9513 - val_loss: 0.6200 - val_recall_m: 0.8590 - val_precision_m: 0.8590 - val_f1_m: 0.8590 - val_single_class_accuracy: 0.3679 - val_single_class_recall: 0.8588

Epoch 00088: val_single_class_accuracy did not improve from 0.41000
Epoch 89/100
Learning rate:  1e-08
28548/28548 [==============================] - 140s 5ms/step - loss: 0.5196 - recall_m: 0.9196 - precision_m: 0.9196 - f1_m: 0.9196 - single_class_accuracy: 0.8313 - single_class_recall: 0.9531 - val_loss: 0.6255 - val_recall_m: 0.8578 - val_precision_m: 0.8578 - val_f1_m: 0.8578 - val_single_class_accuracy: 0.3659 - val_single_class_recall: 0.8600

Epoch 00089: val_single_class_accuracy did not improve from 0.41000
Epoch 90/100
Learning

In [34]:
model = keras.models.load_model('models/ICC_ResNet20v2_model.001.h5')

In [ ]:
y_pred_ = y_pred.argmax(axis=1)
classification_report(y_test, y_pred, output_dict=True)

In [18]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, Reshape#, MaxPooling1D
from keras.callbacks import ReduceLROnPlateau
from keras.layers.noise import GaussianNoise
#Hyperparameters for the network
DENSE = 256
DROPOUT = 0.5
C1_K  = 8 #Number of kernels/feature extractors for first layer
C1_S  = 3 #Width of the convolutional mini networks
C2_K  = 8
C2_S  = 3
C3_K  = 4
C3_S  = 3
C4_K  = 1
C4_S  = 3

activation='relu'

input_dim = X_train.shape[1]

#The model
def make_model():
    model = Sequential()
    #Adding a bit of GaussianNoise also works as regularization
    model.add(GaussianNoise(0, input_shape=(input_dim,)))
    #First two is number of filter + kernel size
    model.add(Reshape((input_dim, 1)))
    model.add(Conv1D(C1_K, (C1_S), activation=activation, border_mode="same"))
    model.add(Conv1D(C2_K, (C2_S), border_mode="same", activation=activation))
    model.add(Conv1D(C3_K, (C3_S), border_mode="same", activation=activation))
    model.add(Conv1D(C4_K, (C4_S), border_mode="same", activation=activation))
    model.add(Flatten())
    model.add(Dropout(DROPOUT))
    model.add(Dense(DENSE, activation=activation))
    model.add(Dense(2, activation='softmax'))

    model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adadelta(lr=1e-3),
                  metrics=[recall_m,precision_m,f1_m,single_class_accuracy,single_class_recall])#, metrics=['mean_absolute_error'])

    return model

In [19]:
#keras.backend.clear_session()
model = make_model()
print(model.summary())

C:\Users\Richard\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(8, 3, activation="relu", padding="same")`
C:\Users\Richard\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(8, 3, activation="relu", padding="same")`
C:\Users\Richard\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(4, 3, activation="relu", padding="same")`
C:\Users\Richard\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(1, 3, activation="relu", padding="same")`
W0117 09:04:15.958045  9492 deprecation.py:506] From C:\Users\Richard\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for u

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gaussian_noise_1 (GaussianNo (None, 12001)             0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 12001, 1)          0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 12001, 8)          32        
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 12001, 8)          200       
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 12001, 4)          100       
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 12001, 1)          13        
_________________________________________________________________
flatten_2 (Flatten)          (None, 12001)             0         
__________

In [20]:
rdlr = ReduceLROnPlateau( min_lr=1e-6, monitor='val_single_class_accuracy', verbose=1,factor=0.8,patience=20)
save_dir = os.path.join(os.getcwd(), 'models')
model_name = 'abd_CNN_model.{epoch:03d}.h5' 
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_single_class_accuracy',
                             verbose=1,
                             save_best_only=True)

model.fit(X_train, Y_train_, epochs=200, batch_size=128, validation_data=(X_test, Y_test_), callbacks=[rdlr,checkpoint],
         class_weight={0:1,1:scale_weight})

Train on 28548 samples, validate on 13627 samples
Epoch 1/200
28548/28548 [==============================] - 12s 431us/step - loss: 0.9242 - recall_m: 0.5603 - precision_m: 0.5604 - f1_m: 0.5604 - single_class_accuracy: 0.3521 - single_class_recall: 0.3801 - val_loss: 0.6931 - val_recall_m: 0.5987 - val_precision_m: 0.5988 - val_f1_m: 0.5988 - val_single_class_accuracy: 0.0970 - val_single_class_recall: 0.4469

Epoch 00001: val_single_class_accuracy improved from -inf to 0.09701, saving model to D:\Work\projects\DL for MS\Notebooks\models\abd_CNN_model.001.h5
Epoch 2/200
28548/28548 [==============================] - 8s 294us/step - loss: 0.9241 - recall_m: 0.5337 - precision_m: 0.5338 - f1_m: 0.5338 - single_class_accuracy: 0.3722 - single_class_recall: 0.5753 - val_loss: 0.6932 - val_recall_m: 0.3686 - val_precision_m: 0.3686 - val_f1_m: 0.3686 - val_single_class_accuracy: 0.1013 - val_single_class_recall: 0.8164

Epoch 00002: val_single_class_accuracy improved from 0.09701 to 0.1013

KeyboardInterrupt: 

In [1]:
plt.plot(model.history.history['val_single_class_accuracy'], label='true positive')
plt.plot(model.history.history['val_recall_m'], label='val_recall')

plt.ylabel('')
plt.xlabel('Epochs')
plt.legend()
# ax2 = plt.gca().twinx()
# ax2.plot(model.history.history['lr'], color='r')
# ax2.set_ylabel('lr',color='r')

_ = plt.legend()

NameError: name 'plt' is not defined

In [12]:
# Load model trained on ABD dataset

dependencies = {
    'recall_m': recall_m,'precision_m':precision_m,'f1_m':f1_m,'single_class_accuracy':single_class_accuracy,
    'single_class_recall':single_class_recall
}
model = keras.models.load_model('models/abd_CNN_model.001.h5',custom_objects=dependencies)

In [13]:
y_pred = model.predict_classes(x_test)
classification_report(y_test.argmax(axis=1), y_pred, output_dict=True)

{'0': {'precision': 0.9529284164859002,
  'recall': 0.33177252473378144,
  'f1-score': 0.49218531174724106,
  'support': 26482},
 '1': {'precision': 0.05581047913776545,
  'recall': 0.7067567567567568,
  'f1-score': 0.10345168628226684,
  'support': 1480},
 'accuracy': 0.3516200557900007,
 'macro avg': {'precision': 0.5043694478118328,
  'recall': 0.5192646407452691,
  'f1-score': 0.29781849901475393,
  'support': 27962},
 'weighted avg': {'precision': 0.9054448835742616,
  'recall': 0.3516200557900007,
  'f1-score': 0.47161003938874874,
  'support': 27962}}

In [ ]:
# # LOADING second dataset
# df_abd=pd.read_pickle('Orbitrap_aplysia/buc.pkl')
# psm_ID=list(pd.read_csv('Aplysia_ganglia/Buccal/DB search psm.csv')['Scan'])
# y=np.zeros(df_abd.shape[0])

# for i in range(0,df_abd.shape[0]):
#     if df_abd.index[i] in psm_ID:
#         y[i]=1

# X = df_abd.drop('RT',axis=1).fillna(0).values
# y =y.astype(int)[X.sum(axis=1)!=0]
# X = X[X.sum(axis=1)!=0]
# X = X/X.max(axis=1).reshape(X.shape[0],1)

# del df_abd
# X = X.reshape((X.shape[0],X.shape[1]))
# x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
# del X

In [20]:
# rdlr = ReduceLROnPlateau( min_lr=1e-4, monitor='val_single_class_accuracy', verbose=1,factor=0.8,patience=20)
# save_dir = os.path.join(os.getcwd(), 'models')
# model_name = 'buc_CNN_model.{epoch:03d}.h5' 
# if not os.path.isdir(save_dir):
#     os.makedirs(save_dir)
# filepath = os.path.join(save_dir, model_name)

# # Prepare callbacks for model saving and for learning rate adjustment.
# checkpoint = ModelCheckpoint(filepath=filepath,
#                              monitor='val_single_class_accuracy',
#                              verbose=1,
#                              save_best_only=True)

# model.fit(x_train, y_train, epochs=100, batch_size=128, validation_data=(x_test, y_test), callbacks=[rdlr,checkpoint],
#          class_weight={0:1,1:scale_weight})

Train on 32677 samples, validate on 14005 samples
Epoch 1/100
32677/32677 [==============================] - 26s 804us/step - loss: 1.3185 - recall_m: 0.9561 - precision_m: 0.9561 - f1_m: 0.9561 - single_class_accuracy: 0.1658 - val_loss: 0.6697 - val_recall_m: 0.9508 - val_precision_m: 0.9508 - val_f1_m: 0.9508 - val_single_class_accuracy: 0.3039

Epoch 00001: val_single_class_accuracy improved from -inf to 0.30391, saving model to D:\Work\projects\DL for MS\models\buc_CNN_model.001.h5
Epoch 2/100
32677/32677 [==============================] - 18s 550us/step - loss: 1.3170 - recall_m: 0.9560 - precision_m: 0.9560 - f1_m: 0.9560 - single_class_accuracy: 0.2288 - val_loss: 0.6677 - val_recall_m: 0.9514 - val_precision_m: 0.9514 - val_f1_m: 0.9514 - val_single_class_accuracy: 0.3097

Epoch 00002: val_single_class_accuracy improved from 0.30391 to 0.30970, saving model to D:\Work\projects\DL for MS\models\buc_CNN_model.002.h5
Epoch 3/100
32677/32677 [==============================] - 18s 

KeyboardInterrupt: 

In [9]:
#data = data[:,1:].reshape((data.shape[0],100,120,1))
#x_train, x_test, y_train, y_test = train_test_split(data, label, test_size=0.3, random_state=1)
del data

In [13]:
x_train.shape

(65244, 100, 120, 1)

In [15]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, Reshape, MaxPooling2D#, MaxPooling1D
from keras.callbacks import ReduceLROnPlateau
from keras.layers.noise import GaussianNoise
#Hyperparameters for the network


input_dim = X_train.shape[:]

#The model
def make_model():
    model = Sequential()
    #convolutional layer with rectified linear unit activation
    model.add(Conv2D(8, kernel_size=(16, 3),
                     activation='relu',
                     input_shape=input_shape))
    #32 convolution filters used each of size 3x3
    #again
    model.add(Conv2D(8, (1, 3), activation='relu'))
    model.add(Conv2D(8, (1, 3), activation='relu'))
    #64 convolution filters used each of size 3x3
    #choose the best features via pooling
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #randomly turn neurons on and off to improve convergence
    model.add(Dropout(0.25))
    #flatten since too many dimensions, we only want a classification output
    model.add(Flatten())
    #fully connected to get all relevant data
    model.add(Dense(512, activation='relu'))
    #one more dropout for convergence' sake :) 
    model.add(Dropout(0.5))
    #output a softmax to squash the matrix into output probabilities
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adadelta(lr=1e-3),
                  metrics=[recall_m,precision_m,f1_m,single_class_accuracy,single_class_recall])#, metrics=['mean_absolute_error'])

    return model

In [16]:
model = make_model()
print(model.summary())

ValueError: Input 0 is incompatible with layer conv2d_2: expected ndim=4, found ndim=3

In [91]:
rdlr = ReduceLROnPlateau( min_lr=1e-6, monitor='val_single_class_accuracy', verbose=1,factor=0.5,patience=20)
save_dir = os.path.join(os.getcwd(), 'models')
model_name = 'abd_buc_CNN2D_model.{epoch:03d}.h5' 
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_single_class_accuracy',
                             verbose=1,
                             save_best_only=True)

model.fit(x_train, y_train, epochs=200, batch_size=128, validation_data=(x_test, y_test), callbacks=[rdlr,checkpoint],
         class_weight={0:1,1:scale_weight})

Train on 65244 samples, validate on 27962 samples
Epoch 1/200
65244/65244 [==============================] - 21s 327us/step - loss: 1.3129 - recall_m: 0.4690 - precision_m: 0.4690 - f1_m: 0.4690 - single_class_accuracy: 0.0540 - single_class_recall: 0.5389 - val_loss: 0.6936 - val_recall_m: 0.4779 - val_precision_m: 0.4779 - val_f1_m: 0.4779 - val_single_class_accuracy: 0.0620 - val_single_class_recall: 0.6181

Epoch 00001: val_single_class_accuracy improved from -inf to 0.06198, saving model to D:\Work\projects\DL for MS\models\abd_buc_CNN2D_model.001.h5
Epoch 2/200
65244/65244 [==============================] - 19s 295us/step - loss: 1.3121 - recall_m: 0.5247 - precision_m: 0.5247 - f1_m: 0.5247 - single_class_accuracy: 0.0613 - single_class_recall: 0.5570 - val_loss: 0.6929 - val_recall_m: 0.5737 - val_precision_m: 0.5737 - val_f1_m: 0.5737 - val_single_class_accuracy: 0.0733 - val_single_class_recall: 0.5966

Epoch 00002: val_single_class_accuracy improved from 0.06198 to 0.07331, 

KeyboardInterrupt: 